In [21]:
import pandas as pd

df = pd.read_csv('escooter_data.csv')
df = df.dropna(subset=['Start Community Area Number', 'End Community Area Number'])
df

,Trip ID,Start Time,End Time,Trip Distance,Trip Duration,Accuracy,Start Census Tract,End Census Tract,Start Community Area Number,End Community Area Number,Start Community Area Name,End Community Area Name,Start Centroid Latitude,Start Centroid Longitude,Start Centroid Location,End Centroid Latitude,End Centroid Longitude,End Centroid Location
10291,0dd95046-1bc8-4464-ba39-7f9e92c668b3,08/06/2019 06:00:00 PM,08/06/2019 06:00:00 PM,1004,236,152,1.703120e+10,1.703120e+10,20.0,20.0,HERMOSA,HERMOSA,41.935048,-87.737604,POINT (-87.7376041954734 41.9350481285613),41.935048,-87.737604,POINT (-87.7376041954734 41.9350481285613)
10293,6819df6a-9623-4ae0-b1d3-ec617197d229,07/08/2019 04:00:00 PM,07/08/2019 05:00:00 PM,9124,3182,10,1.703115e+10,1.703119e+10,15.0,19.0,PORTAGE PARK,BELMONT CRAGIN,41.949592,-87.761958,POINT (-87.7619580301513 41.9495915671753),41.920143,-87.760827,POINT (-87.7608269556143 41.920143229102)
10342,aa71bb53-3e7f-4964-b6e6-030800419346,08/10/2019 09:00:00 PM,08/10/2019 09:00:00 PM,502,97,152,1.703184e+10,1.703184e+10,27.0,27.0,EAST GARFIELD PARK,EAST GARFIELD PARK,41.885045,-87.713862,POINT (-87.71386248059811 41.8850448415253),41.885045,-87.713862,POINT (-87.71386248059811 41.8850448415253)
10367,5d256770-1a8a-5e20-8f78-316d1819f46b,06/25/2019 07:00:00 PM,06/25/2019 08:00:00 PM,5053,2167,0,1.703122e+10,1.703119e+10,22.0,19.0,LOGAN SQUARE,BELMONT CRAGIN,41.921126,-87.699754,POINT (-87.6997540616386 41.9211256768265),41.927819,-87.751362,POINT (-87.7513623675995 41.9278186319205)
10369,dffe5324-5f6f-48e7-9edf-d51876ba7d2f,09/19/2019 08:00:00 AM,09/19/2019 08:00:00 AM,790,111,152,1.703119e+10,1.703120e+10,19.0,20.0,BELMONT CRAGIN,HERMOSA,41.935218,-87.744389,POINT (-87.7443888733072 41.9352176386575),41.927852,-87.735628,POINT (-87.7356282729632 41.9278522690551)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710834,536f855f-2530-4c07-b93e-4de52e75cf1b,09/06/2019 07:00:00 AM,09/06/2019 07:00:00 AM,2524,500,10,NaN,NaN,19.0,19.0,BELMONT CRAGIN,BELMONT CRAGIN,41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187),41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187)
710835,88721836-1a6c-4ae6-b320-8e9ce1861d16,07/11/2019 11:00:00 AM,07/11/2019 11:00:00 AM,2475,480,10,NaN,NaN,20.0,19.0,HERMOSA,BELMONT CRAGIN,41.924348,-87.734740,POINT (-87.7347401601846 41.9243477570064),41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187)
710836,3f930717-06cf-499d-b390-a45282851347,09/17/2019 05:00:00 PM,09/17/2019 05:00:00 PM,2228,590,10,NaN,NaN,19.0,19.0,BELMONT CRAGIN,BELMONT CRAGIN,41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187),41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187)
710837,11d01331-0a12-607e-11d0-13310a12607e,06/21/2019 05:00:00 PM,06/21/2019 05:00:00 PM,1303,1088,10,NaN,NaN,19.0,19.0,BELMONT CRAGIN,BELMONT CRAGIN,41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187),41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187)


In [22]:
df['Start Time'] = pd.to_datetime(df['Start Time'], format="%m/%d/%Y %I:%M:%S %p")
df['End Time'] = pd.to_datetime(df['End Time'], format="%m/%d/%Y %I:%M:%S %p")
df['Start Hour'] = df['Start Time'].dt.hour
df['Same Start End Community'] = df['Start Community Area Name'] == df['End Community Area Name']
unique_areas = pd.unique(df[['Start Community Area Name', 'End Community Area Name']].values.ravel('K'))
all_combinations = pd.MultiIndex.from_product([range(24), unique_areas], names=['Start Hour', 'Start Community Area Name']).to_frame(index=False)

grouped = df.groupby(['Start Hour', 'Start Community Area Name']).agg(
    Average_Trip_Distance=('Trip Distance', 'mean'),
    Average_Trip_Duration=('Trip Duration', 'mean'),
    Trip_Count=('Trip ID', 'count'),
    Same_Start_End_Percentage=('Same Start End Community', lambda x: (x.sum() / x.count()) * 100)
).reset_index()
result = pd.merge(all_combinations, grouped, on=['Start Hour', 'Start Community Area Name'], how='left')
result.fillna(0, inplace=True)
result['Price'] = 0.45
result.to_csv('base_demand.csv', index=None)
result

,Start Hour,Start Community Area Name,Average_Trip_Distance,Average_Trip_Duration,Trip_Count,Same_Start_End_Percentage,Price
0,0,HERMOSA,3454.583333,1040.604167,96.0,53.125000,0.45
1,0,PORTAGE PARK,3229.326087,862.108696,46.0,47.826087,0.45
2,0,EAST GARFIELD PARK,2693.625000,783.125000,32.0,43.750000,0.45
3,0,LOGAN SQUARE,2339.309677,761.180645,775.0,69.548387,0.45
4,0,BELMONT CRAGIN,3075.163636,1145.718182,220.0,75.000000,0.45
...,...,...,...,...,...,...,...
979,23,NORWOOD PARK,0.000000,0.000000,0.0,0.000000,0.45
980,23,WEST ENGLEWOOD,0.000000,0.000000,0.0,0.000000,0.45
981,23,WEST RIDGE,0.000000,0.000000,0.0,0.000000,0.45
982,23,ALBANY PARK,0.000000,0.000000,0.0,0.000000,0.45


In [23]:
supply = result.copy()
supply['Needed_Scooters'] = supply['Trip_Count'] * (1 - (supply['Same_Start_End_Percentage'] / 100)) + 1
supply.to_csv('base_supply.csv', index=None)
supply

,Start Hour,Start Community Area Name,Average_Trip_Distance,Average_Trip_Duration,Trip_Count,Same_Start_End_Percentage,Price,Needed_Scooters
0,0,HERMOSA,3454.583333,1040.604167,96.0,53.125000,0.45,46.0
1,0,PORTAGE PARK,3229.326087,862.108696,46.0,47.826087,0.45,25.0
2,0,EAST GARFIELD PARK,2693.625000,783.125000,32.0,43.750000,0.45,19.0
3,0,LOGAN SQUARE,2339.309677,761.180645,775.0,69.548387,0.45,237.0
4,0,BELMONT CRAGIN,3075.163636,1145.718182,220.0,75.000000,0.45,56.0
...,...,...,...,...,...,...,...,...
979,23,NORWOOD PARK,0.000000,0.000000,0.0,0.000000,0.45,1.0
980,23,WEST ENGLEWOOD,0.000000,0.000000,0.0,0.000000,0.45,1.0
981,23,WEST RIDGE,0.000000,0.000000,0.0,0.000000,0.45,1.0
982,23,ALBANY PARK,0.000000,0.000000,0.0,0.000000,0.45,1.0


In [24]:
supply.columns

Index(['Start Hour', 'Start Community Area Name', 'Average_Trip_Distance',
       'Average_Trip_Duration', 'Trip_Count', 'Same_Start_End_Percentage',
       'Price', 'Needed_Scooters'],
      dtype='object')

In [15]:
df['End Hour'] = df['End Time'].dt.hour
unique_areas = pd.unique(df[['Start Community Area Name', 'End Community Area Name']].values.ravel('K'))
all_combinations = pd.MultiIndex.from_product([range(24), unique_areas], names=['End Hour', 'End Community Area Name']).to_frame(index=False)
end_area_counts = df.groupby(['End Community Area Name', 'End Hour']).size().reset_index(name='Trip Count')
result = pd.merge(all_combinations, end_area_counts, on=['End Hour', 'End Community Area Name'], how='left')
total_trips_per_end_hour = df.groupby('End Hour').size()
result['Average Fraction'] = result.apply(lambda row: row['Trip Count'] / total_trips_per_end_hour[row['End Hour']] if row['End Hour'] in total_trips_per_end_hour else 0, axis=1)
result['Average Fraction'].fillna(0, inplace=True)
result.to_csv('transition_model.csv', index=None)
result

,End Hour,End Community Area Name,Trip Count,Average Fraction
0,0,HERMOSA,110.0,0.022541
1,0,PORTAGE PARK,65.0,0.013320
2,0,EAST GARFIELD PARK,46.0,0.009426
3,0,LOGAN SQUARE,851.0,0.174385
4,0,BELMONT CRAGIN,246.0,0.050410
...,...,...,...,...
979,23,NORWOOD PARK,NaN,0.000000
980,23,WEST ENGLEWOOD,NaN,0.000000
981,23,WEST RIDGE,NaN,0.000000
982,23,ALBANY PARK,NaN,0.000000
